In [96]:
import pandas as pd     
import numpy as np 
import glob as glob
#Importamos las librerias

In [97]:
Base=[]
csv_files = glob.glob('Data/*.csv')
for filename in csv_files:
    data = pd.read_csv(filename,sep=";")
    Base.append(data)

In [98]:
EAM08,EAM09,EMA10,EMA11,EMA12,EMA13,EMA14,EMA15,EMA16,EMA17,EMA18,EMA19,EMA20 = Base
Data_Merge= [EAM08,EAM09,EMA10,EMA11,EMA12,EMA13,EMA14,EMA15,EMA16,EMA17,EMA18,EMA19,EMA20]
#Asignamos cada dataframe a una variable

In [99]:
#Verificación de datos nulos tiempo de demora 2m 21.4s

for i in Data_Merge:
        for col in i.columns:
            missing += i[col].isna().sum()
        for row in i.index:
            missing += i.loc[row,:].isna().sum()

print(missing)


0
